# Visualization Input Data 
### OBS :: Its not interactive since you can zoom. But its a static plots you can use widgets on. 

In [ ]:
import glob
#from sclouds import helpers
import xarray as xr

In [ ]:
path = '/home/hanna/Desktop/master_thesis/era/'

In [ ]:
files = glob.glob(path + "*.nc")

In [ ]:
mapping =  {"r":'relative_humidity_Europe.nc',
            "q":'specific_humidity_Europe.nc',
            "tcc":'total_cloud_cover_Europa.nc',
            "sp":'surface_pressure_Europa.nc',
            "t2m":'temperature_2m_Europa.nc'}

In [ ]:
xr.open_dataset(files[0])

In [ ]:
def get_season(data, season):
    """ OBS: only usefull after merge ts.
    Returns the xarray dataset containing only one season.
    """
    data = data.groupby('time.season')
    for group in data:
        key, dataset = group
        if key == season:
            return dataset

# Timeslider skal være av forskjellig lengde for sesong

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr

# Timeslider 
timeslider = widgets.IntSlider(
    value =7,
    min = 0, 
    max = 10,
    step = 1,
    description = 'Timeslider:',
    disabled = False,
    continuous_update = False,
    orientation = 'horizontal',
    readout = True,
    readout_format = 'd'
)

# Widget for variables
select_variable = widgets.Dropdown(
    options=['r', 'q', 'tcc', 'sp', 't2m'],
    value='t2m', 
    description='Variable:',
    disabled=False,
)

# Widget for season
select_season = widgets.Dropdown(
    options=['DJF', 'MAM', 'JJA', 'SON', 'all'],
    value='all', 
    description='Season:',
    disabled=False,
)

# Widget for season
select_p_lev = widgets.Dropdown(
    #  [300, 400, 500, 700, 850, 1000]
    # [0,1,2,3,4,5]
    options=['1000hPa', '850hPa', '700hPa', '500hPa', '400hPa','300hPa'],
    value='1000hPa', # Think this is init
    description='Pressure lev:',
    disabled=False,
)

def generate_plot(var, ts, season, p_lev):
    """ Creating a static plot. """
    pp = {'1000hPa':0, '850hPa':1, '700hPa':2, '500hPa':3, '400hPa':4,'300hPa':5}
    
    # The change is the new variable
    filename = path + mapping[var]
    fig = plt.figure(figsize=[15,10])
    ax = fig.add_subplot(111, projection = ccrs.Orthographic(central_longitude=20, central_latitude=40))

    # TODO the 
    dset = xr.open_dataset(filename)
   
    if season != "all":
        dset = get_season(dset, season)
        
    if var == "r" or var == "q":
        # Humidities are multilevel parameters.
        dset[var][ts][pp[p_lev]][:][:].plot(ax=ax,cmap='jet', transform=ccrs.PlateCarree())
    else:
        # Surface variables is single level parameters.
        dset[var][ts][:][:].plot(ax=ax,cmap='jet', transform=ccrs.PlateCarree())
    ax.coastlines()
    return fig

In [ ]:
interact_plot = widgets.interact(generate_plot, 
                                 ts = timeslider, 
                                 season = select_season, 
                                 var = select_variable, 
                                 p_lev = select_p_lev);